In [1]:
import numpy as np

In [2]:
def calc_entropy(prob_array):
    if sum(prob_array) != 1:
        return
    return -sum(i * np.log2(i) for i in prob_array)

In [3]:
# calc_entropy([0.17, 0.08, 0.13, 0.62])
# calc_entropy([0.08, 0.12, 0.8])
# calc_entropy([0.17, 0.1, 0.1, 0.63])
# calc_entropy([0.16, 0.03, 0.81])
# calc_entropy([0.02, 0.19, 0.1, 0.69])
# calc_entropy([0.06, 0.3, 0.64])
# calc_entropy([0.23, 0.08, 0.01, 0.68])
calc_entropy([0.02, 0.1, 0.88])

0.6073635558851669

### ハフマン符号を計算してみる

In [4]:
def get_min_syms(d_inv, n):
    """
    下からn番目までの記号を取り出す?
    """
    min_prob = min(d_inv)
    min_prob_syms = d_inv.pop(min_prob)
    
    rem_len = n - len(min_prob_syms)
    # オーバー
    if rem_len < 0:
        min_prob_syms.sort(reverse=True)
        return min_prob_syms[:n]
    elif rem_len == 0:
        return min_prob_syms
    else:
        # 足りない場合は再帰
        return min_prob_syms + get_min_syms(d_inv, rem_len)

def create_connection_dict(prob_d):
    prob_d_cp = prob_d.copy()
    cnct_d = {}
    next_key = max(prob_d_cp) + 1

    while len(prob_d_cp) > 1:
        prob_d_inv = {}
        # 逆引き辞書作成
        for k, v in prob_d_cp.items():
            if v in prob_d_inv:
                prob_d_inv[v].append(k)
            else:
                prob_d_inv[v] = [k]

        print(prob_d_inv)

        min_syms = get_min_syms(prob_d_inv, 2)

        prob_d_cp[next_key] = sum(v for k, v in prob_d_cp.items() if k in min_syms)
        cnct_d[next_key] = sorted(min_syms)
        
        for i in min_syms:
            prob_d_cp.pop(i)
    
        print(prob_d_cp)
        print(cnct_d)
        next_key += 1

    return cnct_d

def generate_code(cnct_d, code_d):
    """
    枝の繋がりから符号を作成
    """
    max_key = max(cnct_d)
    children = cnct_d.pop(max_key)
    
    print(cnct_d)
    
    for i, j in enumerate(children):
        if max_key in code_d:
            code_d[j] = code_d[max_key] + str(i)
        else:
            code_d[j] = str(i)
    
    if max_key in code_d:
        code_d.pop(max_key)
    print(code_d)
    
    if any(cnct_d):
        generate_code(cnct_d, code_d)

In [5]:
# prob_arr = np.array([0.2, 0.19, 0.07, 0.05, 0.49])
# prob_arr = np.array([0.05, 0.05, 0.05, 0.05, 0.8])
prob_arr = np.array([0.12, 0.01, 0.22, 0.07, 0.02, 0.56])
print(prob_arr.sum())
prob_dict = {k: v for k, v in enumerate(prob_arr)}

connect_dict = create_connection_dict(prob_dict)

print("#" * 100)

code_dict = {}
generate_code(connect_dict, code_dict)

sorted_keys = sorted(code_dict.keys())
print("answer: ", end="")
for i in sorted_keys:
    print(code_dict[i], end=" ")
print()

1.0
{0.12: [0], 0.01: [1], 0.22: [2], 0.07: [3], 0.02: [4], 0.56: [5]}
{0: 0.12, 2: 0.22, 3: 0.07, 5: 0.56, 6: 0.03}
{6: [1, 4]}
{0.12: [0], 0.22: [2], 0.07: [3], 0.56: [5], 0.03: [6]}
{0: 0.12, 2: 0.22, 5: 0.56, 7: 0.1}
{6: [1, 4], 7: [3, 6]}
{0.12: [0], 0.22: [2], 0.56: [5], 0.1: [7]}
{2: 0.22, 5: 0.56, 8: 0.22}
{6: [1, 4], 7: [3, 6], 8: [0, 7]}
{0.22: [2, 8], 0.56: [5]}
{5: 0.56, 9: 0.44}
{6: [1, 4], 7: [3, 6], 8: [0, 7], 9: [2, 8]}
{0.56: [5], 0.44: [9]}
{10: 1.0}
{6: [1, 4], 7: [3, 6], 8: [0, 7], 9: [2, 8], 10: [5, 9]}
####################################################################################################
{6: [1, 4], 7: [3, 6], 8: [0, 7], 9: [2, 8]}
{5: '0', 9: '1'}
{6: [1, 4], 7: [3, 6], 8: [0, 7]}
{5: '0', 2: '10', 8: '11'}
{6: [1, 4], 7: [3, 6]}
{5: '0', 2: '10', 0: '110', 7: '111'}
{6: [1, 4]}
{5: '0', 2: '10', 0: '110', 3: '1110', 6: '1111'}
{}
{5: '0', 2: '10', 0: '110', 3: '1110', 1: '11110', 4: '11111'}
answer: 110 11110 10 1110 11111 0 


### ランレングス・ハフマン符号

In [31]:
def run_length_each_prob(Pb, N):
    """
    各長さの確率
    """
    return np.array([(1 - Pb) ** i * Pb for i in range(N)] + [(1 - Pb) ** N])

def calc_source_len_mean(Pb, N):
    return (1 - (1 - Pb) ** N) / Pb

In [34]:
Pb = 0.007
N = 4

prob_arr = run_length_each_prob(Pb, N)
print(prob_arr, prob_arr.sum())

prob_dict = {k: v for k, v in enumerate(prob_arr)}

connect_dict = create_connection_dict(prob_dict)

print("#" * 100)

code_dict = {}
generate_code(connect_dict, code_dict)

len_arr = np.array([len(code_dict[k]) for k in sorted(code_dict.keys())])

codeword_len_mean = np.dot(prob_arr, len_arr)
print("L = %f" % codeword_len_mean)
source_len_mean = calc_source_len_mean(Pb, N)
print("n_N = %f" % source_len_mean)

print("l_N = %f" % (codeword_len_mean / source_len_mean))

[0.007      0.006951   0.00690234 0.00685403 0.97229263] 1.0
{0.007: [0], 0.006951: [1], 0.006902343: [2], 0.006854026599000001: [3], 0.972292630401: [4]}
{0: 0.007, 1: 0.006951, 4: 0.972292630401, 5: 0.013756369599}
{5: [2, 3]}
{0.007: [0], 0.006951: [1], 0.972292630401: [4], 0.013756369599: [5]}
{4: 0.972292630401, 5: 0.013756369599, 6: 0.013951}
{5: [2, 3], 6: [0, 1]}
{0.972292630401: [4], 0.013756369599: [5], 0.013951: [6]}
{4: 0.972292630401, 7: 0.027707369598999998}
{5: [2, 3], 6: [0, 1], 7: [5, 6]}
{0.972292630401: [4], 0.027707369598999998: [7]}
{8: 1.0}
{5: [2, 3], 6: [0, 1], 7: [5, 6], 8: [4, 7]}
####################################################################################################
{5: [2, 3], 6: [0, 1], 7: [5, 6]}
{4: '0', 7: '1'}
{5: [2, 3], 6: [0, 1]}
{4: '0', 5: '10', 6: '11'}
{5: [2, 3]}
{4: '0', 5: '10', 0: '110', 1: '111'}
{}
{4: '0', 0: '110', 1: '111', 2: '100', 3: '101'}
L = 1.055415
n_N = 3.958196
l_N = 0.266640


### ZL符号

In [97]:
# symbol_series = "AAAAAAAAAAAABBAAAABA"
# symbol_series = "ABAAABAAABABBBABABAB"
# symbol_series = "ABAAABBAAAABBABABAAA"
symbol_series = "AAAAAAAABBBABABBAABB"

codes = []
known_patterns = {}

i = 0
while i < len(symbol_series):
    for j in range(i):
        for l in range(1, i - j + 1):
            pattern = symbol_series[j:j+l]
            if pattern not in known_patterns:
                known_patterns[pattern] = (j, l)
#                 print(i, j, l, pattern)
    for j in range(i, 0, -1):
        pattern = symbol_series[i:i + j]
        print(i, j, pattern)
        if pattern in known_patterns:
            if i + j >= len(symbol_series):
                print("末尾余り")
                i = float("inf")
            else:
                codes.append(known_patterns[pattern] + (symbol_series[i+j],))
                i += j + 1
            break
    else:
        codes.append((0, 0, symbol_series[i]))
        i += 1
    print("#" * 100)
    
# print(known_patterns)
print(codes)

for i in codes:
    print(i[1] + 1, end=" ")
print()

####################################################################################################
1 1 A
####################################################################################################
3 3 AAA
####################################################################################################
7 7 ABBBABA
7 6 ABBBAB
7 5 ABBBA
7 4 ABBB
7 3 ABB
7 2 AB
7 1 A
####################################################################################################
9 9 BBABABBAA
9 8 BBABABBA
9 7 BBABABB
9 6 BBABAB
9 5 BBABA
9 4 BBAB
9 3 BBA
9 2 BB
9 1 B
####################################################################################################
11 11 ABABBAABB
11 10 ABABBAABB
11 9 ABABBAABB
11 8 ABABBAAB
11 7 ABABBAA
11 6 ABABBA
11 5 ABABB
11 4 ABAB
11 3 ABA
11 2 AB
####################################################################################################
14 14 BBAABB
14 13 BBAABB
14 12 BBAABB
14 11 BBAABB
14 10 BBAABB
14 9 BBAABB
14 8 BBAABB
14 7 BBAABB
14

### 実用的なZL

In [138]:
offset_bit = 3
length_bit = 2

symbol_series = "AAABAABAAABB"

offset = 1 << offset_bit
length = 1 << length_bit

buffer = "A" * offset + symbol_series[:length + 1]

symbol_remain = symbol_series[length + 1:]

codes = []
while len(buffer) > offset:
    print(buffer[:offset] + "|" + buffer[offset:])
    print(symbol_remain)
    
    known_patterns = {}
    for i in range(offset):
        for j in range(i + 1, offset + 1):
            pattern = buffer[i:j]
            if pattern not in known_patterns:
                known_patterns[pattern] = (i, j - i)
#     print(known_patterns)
    for l in range(len(buffer) - offset, 0, -1):
        pattern = buffer[offset:offset+l]
        print(i, j, pattern)
        if pattern in known_patterns:
            codes.append(known_patterns[pattern] + (buffer[offset+l],))
            buffer = buffer[l+1:] + symbol_remain[:l+1]
            symbol_remain = symbol_remain[l+1:]
            break
    else:
        codes.append((0, 0, buffer[offset]))
        buffer = buffer[1:] + symbol_remain[:1]
        symbol_remain = symbol_remain[1:]
    print("#" * 100)
print(codes)

for i in codes:
    print(format(i[0], "0%db" % offset_bit) + format(i[1], "0%db" % length_bit) + str((i[2] == "B") * 1), end=" ")
print()

AAAAAAAA|AAABA
ABAAABB
7 8 AAABA
7 8 AAAB
7 8 AAA
####################################################################################################
AAAAAAAB|AABAA
ABB
7 8 AABAA
7 8 AABA
7 8 AAB
####################################################################################################
AAABAABA|AABB

7 8 AABB
7 8 AAB
####################################################################################################
[(0, 3, 'B'), (5, 3, 'A'), (1, 3, 'B')]
000111 101110 001111 


### 実用的なZL (復号)

In [167]:
offset_bit = 4
length_bit = 3

codes = "00000011 11001000 10010111 00000110"
codes = codes.split(" ")
codes = [(int(i[:offset_bit], 2), int(i[offset_bit:offset_bit+length_bit], 2), chr(ord("A") + int(i[-1]))) for i in codes]
codes

offset = 1 << offset_bit
length = 1 << length_bit

buffer = "A" * offset
symbol_series = ""

for i in codes:
    print(i)
    print(buffer[:offset] + "|" + buffer[offset:])
    pattern = buffer[i[0]:i[0]+i[1]] + i[2]
    symbol_series += pattern
    buffer = (buffer + pattern)[i[1]+1:]

print(buffer[:offset] + "|" + buffer[offset:] + "\n")
print(symbol_series)

(0, 1, 'B')
AAAAAAAAAAAAAAAA|
(12, 4, 'A')
AAAAAAAAAAAAAAAB|
(9, 3, 'B')
AAAAAAAAAABAAABA|
(0, 3, 'A')
AAAAAABAAABAABAB|
AABAAABAABABAAAA|

ABAAABAABABAAAA


In [148]:
chr(ord("A") + 1)

'B'